### use llm for topic representation

In [29]:
import os,sys
sys.path.insert(0,'../libs')
from utils import load_json
import openai
from llm_utils import BSAgent_legacy
import numpy as np
from bertopic import BERTopic
import pandas as pd
import re,json,copy
from tqdm import tqdm
## load API Key
key = load_json('/home/chuang/Dev/Keys/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT']['API_KEY']
openai.api_key  = os.getenv('OPENAI_API_KEY')

- load legacy llm agent class, as fund server having problem with the new version

In [30]:
llm_agent  = BSAgent_legacy(model="gpt-3.5-turbo-1106", 
                    temperature=0)

In [31]:
## just run one test, make sure the api works 
pt = {'System':'You are a helpful assistant.',
      'Human':'What is your name?'}
res = llm_agent.get_response_content(prompt_template=pt)
print(res)  


I'm a helpful assistant here to answer your questions and assist you with any information you need. You can call me "Assistant."


In [32]:
res_folder = '/data/chuang/news_scrape/data/news_search_res'
emb_path  = os.path.join(res_folder,'sentence_embeddings.npy')
docs_path = os.path.join(res_folder,'docs.npy')
topic_model_out_path = os.path.join(res_folder,'topic_model')

In [33]:
topic_model = BERTopic.load(topic_model_out_path)
topic_model.calculate_probabilities = False ## do not calculate probs for all topics 
## load training doc and embeddings 
embeddings = np.load(emb_path)
docs = np.load(docs_path)

In [34]:
topic_model.visualize_barchart()

In [35]:
delim = '\n----------------\n----------------\n'

topic_df = topic_model.get_topic_info()
topic_df['topic_words'] = topic_model.generate_topic_labels(nr_words=10,
                                                            topic_prefix=False,
                                                            word_length=50,
                                                            separator=", ")
rep_doc_dict = topic_model.get_representative_docs()      
rep_doc_dict = {k:delim.join([re.sub(r'\n+', '\n', text) for text in v]) for k,v in rep_doc_dict.items()} 
topic_df.head()

,Topic,Count,Name,topic_words
0,-1,4988,-1_2021_2023_2022_2020,"2021, 2023, 2022, 2020, 2019, july, june, radi..."
1,0,3484,0_cricket_indies_west indies_tournament,"cricket, indies, west indies, tournament, tenn..."
2,1,516,1_radio svg_july 2018_july 2023_june 2018,"radio svg, july 2018, july 2023, june 2018, ju..."
3,2,445,2_covid 19_covid_coronavirus_19 cases,"covid 19, covid, coronavirus, 19 cases, flu, q..."
4,3,438,3_accused_prosecution_robbery_offences court,"accused, prosecution, robbery, offences court,..."


In [36]:
summary_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
Please summarize the following new article with less then 100 words: 
----------------
----------------

{DOCUMENTS}

----------------
----------------
Please write your summery below:
"""}

topic_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
I have a topic that contains the following documents delimited with ----------------: 
----------------
----------------

{DOCUMENTS}

----------------
----------------

The topic is described by the following keywords: {KEYWORDS}

Based on the information above, extract a short topic label, a list of topic words and a short descrption of the topic.
Make sure the label has no more than 3 words.
Make sure topic words arel less than 10 words.
And also makse sure the description is only a short highlevel sumamry.  
Please respond in clean json format as follow:
topic: <topic label>,
topic_words: [<topic workds>],
description: <topic description>

"""}
#topic_pt['Human'] = topic_pt['Human'].format(DOCUMENTS=rep_doc_dict[1],KEYWORDS=topic_df.iloc[1]['topic_words'])

In [37]:
llm_res = []
for k,v in tqdm(rep_doc_dict.items()):
    topic_pt_temp = copy.copy(topic_pt)
    topic_pt_temp['Human']=topic_pt_temp['Human'].format(DOCUMENTS=v,KEYWORDS=topic_df.iloc[k]['topic_words'])
    try:
        res = llm_agent.get_response_content(prompt_template=topic_pt_temp,conv_history=[])
        res_dict = llm_agent.parse_load_json_str(res)
    except:
        res_dict = {'topic':None,'topic_words':None,'description':None}
    #print(res_dict['topic'])
    res_dict['topic_id'] = k
    llm_res.append(res_dict)

  0%|          | 0/67 [00:00<?, ?it/s]

Function failed with error HTTP code 504 from API (<HEAD><TITLE>Connection Timed Out</TITLE></HEAD>
<BODY BGCOLOR="white" FGCOLOR="black"><H1>Connection Timed Out</H1><HR>
<FONT FACE="Helvetica,Arial"><B>
Description: Connection Timed Out</B></FONT>
<HR>
<!-- default "Connection Timed Out" response (504) -->
</BODY>
                                                                                                                                                                                                                                                                                                                                                                                                               ). Retrying after 5 seconds...


 21%|██        | 14/67 [01:47<02:26,  2.76s/it] 

Function failed with error HTTP code 504 from API (<HEAD><TITLE>Connection Timed Out</TITLE></HEAD>
<BODY BGCOLOR="white" FGCOLOR="black"><H1>Connection Timed Out</H1><HR>
<FONT FACE="Helvetica,Arial"><B>
Description: Connection Timed Out</B></FONT>
<HR>
<!-- default "Connection Timed Out" response (504) -->
</BODY>
                                                                                                                                                                                                                                                                                                                                                                                                               ). Retrying after 5 seconds...


 28%|██▊       | 19/67 [03:20<07:55,  9.91s/it]

...

 82%|████████▏ | 55/67 [14:25<01:33,  7.81s/it]

Function failed with error HTTP code 504 from API (<HEAD><TITLE>Connection Timed Out</TITLE></HEAD>
<BODY BGCOLOR="white" FGCOLOR="black"><H1>Connection Timed Out</H1><HR>
<FONT FACE="Helvetica,Arial"><B>
Description: Connection Timed Out</B></FONT>
<HR>
<!-- default "Connection Timed Out" response (504) -->
</BODY>
                                                                                                                                                                                                                                                                                                                                                                                                               ). Retrying after 5 seconds...


 94%|█████████▍| 63/67 [15:50<00:17,  4.34s/it]

Function failed with error HTTP code 504 from API (<HEAD><TITLE>Connection Timed Out</TITLE></HEAD>
<BODY BGCOLOR="white" FGCOLOR="black"><H1>Connection Timed Out</H1><HR>
<FONT FACE="Helvetica,Arial"><B>
Description: Connection Timed Out</B></FONT>
<HR>
<!-- default "Connection Timed Out" response (504) -->
</BODY>
                                                                                                                                                                                                                                                                                                                                                                                                               ). Retrying after 5 seconds...


100%|██████████| 67/67 [17:04<00:00, 15.29s/it]


In [38]:
llm_df = pd.DataFrame(llm_res)
llm_df=llm_df.sort_values('topic_id',ascending=True)
llm_df.columns = ['llm_topic','llm_topic_words','llm_description','llm_topic_id']
llm_df.head()

,llm_topic,llm_topic_words,llm_description,llm_topic_id
0,Chocolate Cake Recipe,"[recipe, baking, chocolate, cake, icing, sugar...",A collection of articles discussing chocolate ...,-1
1,Cricket Season 2017,"[Cricket, Season, Competition, Tournament, Pla...",Summary of the 2017 cricket season in St Vince...,0
2,Cricket Tournament,"[cricket, Indies, West Indies, tournament, mat...",Coverage of a cricket tournament in the West I...,1
3,Covid-19 in SVG,"[Covid-19, St. Vincent and the Grenadines, cas...",Updates on the Covid-19 situation in St. Vince...,2
4,Legal Issues in Court,"[court, magistrate, charges, sentencing, bail]",This topic covers various legal issues in cour...,3


In [39]:
final_result = pd.merge(topic_df, llm_df, how='left',left_on='Topic',right_on='llm_topic_id')
final_result.head()

,Topic,Count,Name,topic_words,llm_topic,llm_topic_words,llm_description,llm_topic_id
0,-1,4988,-1_2021_2023_2022_2020,"2021, 2023, 2022, 2020, 2019, july, june, radi...",Chocolate Cake Recipe,"[recipe, baking, chocolate, cake, icing, sugar...",A collection of articles discussing chocolate ...,-1
1,0,3484,0_cricket_indies_west indies_tournament,"cricket, indies, west indies, tournament, tenn...",Cricket Season 2017,"[Cricket, Season, Competition, Tournament, Pla...",Summary of the 2017 cricket season in St Vince...,0
2,1,516,1_radio svg_july 2018_july 2023_june 2018,"radio svg, july 2018, july 2023, june 2018, ju...",Cricket Tournament,"[cricket, Indies, West Indies, tournament, mat...",Coverage of a cricket tournament in the West I...,1
3,2,445,2_covid 19_covid_coronavirus_19 cases,"covid 19, covid, coronavirus, 19 cases, flu, q...",Covid-19 in SVG,"[Covid-19, St. Vincent and the Grenadines, cas...",Updates on the Covid-19 situation in St. Vince...,2
4,3,438,3_accused_prosecution_robbery_offences court,"accused, prosecution, robbery, offences court,...",Legal Issues in Court,"[court, magistrate, charges, sentencing, bail]",This topic covers various legal issues in cour...,3


In [40]:
llm_fintune_out = os.path.join(res_folder,'llm_results.xlsx')
final_result.to_excel(llm_fintune_out)